# CombinedProxies NetCDF Production

The following script is used to produce a NetCDF4 file containing the data from all proxies combined into one file.

In [ ]:
import xarray as xr

Opening SIE data as a base onto which to append.

In [ ]:
folder = '~/Desktop/IMAS/ncfiles/'  #filepath
pfile = 'Proxy_combined_data_v4.nc'  #proxy filename
df = xr.open_dataset(folder+pfile) #.sel(year= slice(startyear, endyear))

#Mean on month as WHG_dex has monthly values
X = df[(list(df.keys())[1:])].mean('month')

Merge in the DSS data. Doing a rename to indicate which proxy each variable in the resultant file came from by adding the proxy name as a prefix.

In [ ]:
s = xr.open_dataset(folder+'DSS_2k_summer_centred.nc')
keys = list(s.keys())
renamed = list(map(lambda x: 'DSS_summer_'+x, keys))
s = s.rename(dict(zip(keys,renamed)))
X = X.merge(s)

s = xr.open_dataset(folder+'DSS_2k_winter_centred.nc')
keys = list(s.keys())
renamed = list(map(lambda x: 'DSS_winter_'+x, keys))
s = s.rename(dict(zip(keys,renamed)))
X = X.merge(s)

Merge in Sarah's three ice cores' data. Need to calculate avg for each year by doing weighted avg from different thickness sections in each year.

In [ ]:
cores = ['TIR18', 'FK17','IC12']
proxies = ['MSA','Na','SO4']
for c in range(3): #3 cores
    cdata = xr.open_dataset(folder+cores[c]+'.nc')        
    for p in range(3): #3 proxies
        proxy = proxies[p]
    
        # Do thickness weighting for the proxy.
        for i in range(len(cdata['topdepth'])):
            cdata[proxy][i] = cdata['thickness'][i]*cdata[proxy][i]
        
        cdata = cdata.rename({proxy: cores[c]+'_'+proxy})
    cdata = cdata.groupby("year").mean()
    cdata = cdata.drop_vars('thickness')
    #Due to cutoff in age model, remove first year as that contains
    # all values beyond the age model and so is incorrect
    cdata = cdata.drop_sel(year=cdata.get_index('year')[0])
    X = X.merge(cdata)

Merge in the MBS data, dropping the sample names and mid depths so we can do an avg for other chemistry on the year dimension.

In [ ]:
mbsdata = ['MBS_Main','MBS_Alpha','MBS_Charlie']
for c in range(3):
    ds = mbsdata[c]
    s = xr.open_dataset(folder+ds+'.nc')    
    s = s.drop_vars(['Mid Depth','Sample name'])
    keys = list(s.keys())
    renamed = list(map(lambda x: ds+'_'+x, keys))
    s = s.rename(dict(zip(keys,renamed))).groupby('year').mean()
    X = X.merge(s)

Merge in Crockart et al data.

In [ ]:
X = X.merge(xr.open_dataset(folder+'Crockart_et_al_2_0.nc'))

Save the combined proxies file.

In [ ]:
CombinedProxies = X
filedirectory = r"C:\Users\Alfie\Desktop\IMAS\ncfiles\CombinedProxies.nc"
CombinedProxies.to_netcdf(filedirectory)